# Benchmark Matrix 

In [1]:
import adelie as ad
import numpy as np
import pandas as pd
import os
from time import time

In [2]:
def bench_naive(X, q=8):
    n, p = X.rows(), X.cols()
    df = pd.DataFrame()

    weights = np.ones(n)
    sqrt_weights = np.sqrt(weights)

    v = np.random.normal(0, 1, n)
    start = time()
    X.cmul(0, v)
    elapsed = time() - start
    df["cmul"] = [elapsed * 1e3]

    v = 1.3
    out = np.empty(n)
    start = time()
    X.ctmul(0, v, weights, out)
    elapsed = time() - start
    df["ctmul"] = [elapsed * 1e3]

    v = np.random.normal(0, 1, n)
    out = np.empty(q)
    start = time()
    X.bmul(0, q, v, out)
    elapsed = time() - start
    df["bmul"] = [elapsed * 1e3]

    v = np.random.normal(0, 1, q)
    out = np.empty(n)
    start = time()
    X.btmul(0, q, v, weights, out)
    elapsed = time() - start
    df["btmul"] = [elapsed * 1e3]

    v = np.random.normal(0, 1, n)
    out = np.empty(p)
    start = time()
    X.mul(v, out)
    elapsed = time() - start
    df["mul"] = [elapsed * 1e3]

    v = np.random.normal(0, 1, n)
    out = np.empty((q, q), order="F")
    buffer = np.empty((n, q), order="F")
    start = time()
    X.cov(0, q, sqrt_weights, out, buffer)
    elapsed = time() - start
    df["cov"] = [elapsed * 1e3]

    return df

## ad.matrix.dense

In [3]:
n = int(2 ** 15)
p = 300
X = np.random.normal(0, 1, (n, p))

In [9]:
bench_naive(ad.matrix.dense(X, method="naive", n_threads=1))

,cmul,ctmul,bmul,btmul,mul,cov
0,0.240803,0.097752,0.287771,0.241756,3.637791,1.302719


In [13]:
bench_naive(ad.matrix.dense(X, method="naive", n_threads=4))

,cmul,ctmul,bmul,btmul,mul,cov
0,0.20647,0.081539,0.112057,0.115395,2.32029,0.501871


## ad.matrix.kronecker_eye

In [14]:
n = int(2 ** 15)
p = 300
K = 4
X = np.random.normal(0, 1, (n, p))

In [24]:
bench_naive(ad.matrix.kronecker_eye(X, K=K, n_threads=4), q=K)

,cmul,ctmul,bmul,btmul,mul,cov
0,0.1297,0.065088,0.063181,0.159264,3.937721,0.253439


In [20]:
bench_naive(ad.matrix.kronecker_eye(ad.matrix.dense(X, n_threads=4), K=K), q=K)

,cmul,ctmul,bmul,btmul,mul,cov
0,0.193357,0.181198,0.22316,0.263929,9.7754,0.605583


## ad.matrix.snp_unphased

In [3]:
n = 10000
p = 100
seed = 0
data = ad.data.hased(n, p, seed=seed)
filename = "/tmp/bench_snp_unphased.snpdat"
handler = ad.io.snp_unphased(filename)
_ = handler.write(data["X"])

In [4]:
bench_naive(ad.matrix.snp_unphased(filename, n_threads=1))

,cmul,ctmul,bmul,btmul,mul,cov
0,0.013113,0.008821,0.044823,0.061035,0.308275,0.766039


In [5]:
bench_naive(ad.matrix.snp_unphased(filename, n_threads=4))

,cmul,ctmul,bmul,btmul,mul,cov
0,0.011921,0.029802,0.119925,0.036955,0.127077,0.39196


In [20]:
os.remove(filename)

## ad.matrix.snp_phased_ancestry

In [3]:
n = 100000
s = 100
A = 8
seed = 0
data = ad.data.snp_phased_ancestry(n, s, A, seed=seed)
filename = "/tmp/bench_snp_phased_ancestry.snpdat"
handler = ad.io.snp_phased_ancestry(filename)
_ = handler.write(data["X"], data["ancestries"], A)

In [7]:
bench_naive(ad.matrix.snp_phased_ancestry(filename, n_threads=1))

,cmul,ctmul,bmul,btmul,mul,cov
0,0.07987,0.109196,0.08297,0.081301,5.756855,0.431299


In [10]:
bench_naive(ad.matrix.snp_phased_ancestry(filename, n_threads=8))

,cmul,ctmul,bmul,btmul,mul,cov
0,0.051022,0.069141,0.09203,0.054121,1.181841,0.426054


In [ ]:
os.remove(filename)